# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [43]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [44]:
clean_city_data = pd.read_csv("clean_city_data.csv")
clean_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kitgum,3.28,32.89,18.34,91,35,1.63,UG,1596317805
1,Bredasdorp,-34.53,20.04,9.00,93,0,2.10,ZA,1596317806
2,Rikitea,-23.12,-134.97,23.29,77,11,3.64,PF,1596317668
3,Iracoubo,5.48,-53.20,26.68,84,90,3.18,GF,1596317669
4,Vao,-22.67,167.48,19.04,69,32,7.60,NC,1596317669
...,...,...,...,...,...,...,...,...,...
546,Dubai,25.26,55.30,36.00,53,38,2.60,AE,1596317731
547,Sur,22.57,59.53,28.79,72,12,5.20,OM,1596317805
548,Lata,40.16,-8.33,19.44,72,0,2.68,PT,1596317614
549,Okhotsk,59.38,143.30,16.23,61,0,1.93,RU,1596317805


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [45]:
# Configure gmaps
#gmaps.configure(api_keys=g_key)
gmaps.configure

# Store latitude and longitude in locations
locations = clean_city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = clean_city_data["Humidity"].astype(float)

# Plot heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating =False, max_intensity=0,
                                point_radius=1)                                                                 
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [46]:

#plt.savefig("Humidity Heatmap.png", dpi=175)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [47]:
# filter cities to a "perfect" temperature between 22 and 27 degrees celsius (70-80 F)
maxtemp_min = clean_city_data.loc[clean_city_data["Max Temp"] > 22]
maxtemp_max = maxtemp_min.loc[maxtemp_min["Max Temp"] < 27]

# filter cities even further for no clouds (close to zero %)
cloud = maxtemp_max.loc[maxtemp_max["Cloudiness"] < 5]

# filter cities even further for windspeeds of 4.5 m/s (less than 10mph)
windspeed = cloud.loc[cloud["Wind Speed"] < 4.5]

# filter cities even further for humidity under 56 %
humidity = windspeed.loc[windspeed["Humidity"] < 56]
humidity

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,Pozo Colorado,-23.49,-58.80,26.33,41,3,2.56,PY,1596317673
85,Fairbanks,64.84,-147.72,25.00,50,1,0.78,US,1596317567
119,Assiniboia,49.63,-105.98,26.01,29,0,3.00,CA,1596317698
421,Ribas do Rio Pardo,-20.44,-53.76,24.35,38,0,3.84,BR,1596317771
465,Mizdah,31.45,12.98,23.81,40,0,2.47,LY,1596317783
485,Lüderitz,-26.65,15.16,24.00,19,0,4.10,NaN,1596317789
506,Morros,-9.45,-46.30,25.22,38,1,2.51,BR,1596317794
508,Changji,44.02,87.32,23.00,33,0,2.00,CN,1596317795
530,Jayanca,-6.39,-79.82,26.67,55,0,2.24,PE,1596317801
535,Beni Suef,29.07,31.10,26.88,41,0,3.99,EG,1596317802


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [48]:
#  rename and add hotel name to dataframe column
hotel_df = humidity
hotel_df["Hotel Name"] = ""
hotel_df.head(15)

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
22,Pozo Colorado,-23.49,-58.80,26.33,41,3,2.56,PY,1596317673,
85,Fairbanks,64.84,-147.72,25.00,50,1,0.78,US,1596317567,
119,Assiniboia,49.63,-105.98,26.01,29,0,3.00,CA,1596317698,
421,Ribas do Rio Pardo,-20.44,-53.76,24.35,38,0,3.84,BR,1596317771,
465,Mizdah,31.45,12.98,23.81,40,0,2.47,LY,1596317783,
485,Lüderitz,-26.65,15.16,24.00,19,0,4.10,NaN,1596317789,
506,Morros,-9.45,-46.30,25.22,38,1,2.51,BR,1596317794,
508,Changji,44.02,87.32,23.00,33,0,2.00,CN,1596317795,
530,Jayanca,-6.39,-79.82,26.67,55,0,2.24,PE,1596317801,
535,Beni Suef,29.07,31.10,26.88,41,0,3.99,EG,1596317802,


In [49]:
#set parameter to search hotel within 5000 meters and hit google places API for call's based on coordinates

#  params dictionary to update each iteration 
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # use search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make reguest
    name = requests.get(base_url, params=params)
    
    # convert to json
    name = name.json()
    
    #print to confirm data
    #print(json.dumps(name, indent=4, sort_keys= True))
     
    # use try and except to skip any missing data points
    try:
        hotel_df.loc[index, "Hotel Name"] = name["results"][0]["name"]
    except:
        print("Missing...skipping")
   

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing...skipping


In [50]:
# confirm hotel name data appears in dataframe
hotel_df.head(15)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
22,Pozo Colorado,-23.49,-58.80,26.33,41,3,2.56,PY,1596317673,Hospedaje la Amistad
85,Fairbanks,64.84,-147.72,25.00,50,1,0.78,US,1596317567,Westmark Fairbanks Hotel & Conference Center
119,Assiniboia,49.63,-105.98,26.01,29,0,3.00,CA,1596317698,Canalta Hotel Assiniboia
421,Ribas do Rio Pardo,-20.44,-53.76,24.35,38,0,3.84,BR,1596317771,Ataíde Martins
465,Mizdah,31.45,12.98,23.81,40,0,2.47,LY,1596317783,محمدالمبروك الاريقط
485,Lüderitz,-26.65,15.16,24.00,19,0,4.10,NaN,1596317789,LÜDERITZ NEST HOTEL
506,Morros,-9.45,-46.30,25.22,38,1,2.51,BR,1596317794,
508,Changji,44.02,87.32,23.00,33,0,2.00,CN,1596317795,Lvye Holiday Resort
530,Jayanca,-6.39,-79.82,26.67,55,0,2.24,PE,1596317801,Rancho Santana Horseback Riding
535,Beni Suef,29.07,31.10,26.88,41,0,3.99,EG,1596317802,Tolip Inn Beni Suef


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [53]:
# Configure gmaps
#gmaps.configure(api_keys=g_key)
gmaps.configure

# Store latitude and longitude in locations
locations = clean_city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = clean_city_data["Humidity"].astype(float)

# Plot heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=0,
                                point_radius=1)                                                                 
# Add layer
fig.add_layer(heat_layer)

# Add marker layer ontop of heat map and marker pop-up boxes
markers = gmaps.marker_layer(marker_locations,
          info_box_content=hotel_info)                  
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))